<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `SOCOTEC` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SOCOTEC`


In [1]:
# Check for GPU availability
!nvidia-smi

Mon Dec  9 18:21:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Install required libraries
!pip install -q transformers torch accelerate datasets openai
!pip install -q bitsandbytes

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.1 MB/s eta 0:00:00


In [3]:
# Implement function library as specified
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"

# Function mapping dictionary
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}

In [4]:
# Create synthetic training dataset
training_data = [
    {"input": "What is the sum of 7 and 3?", "output": "add(7, 3)"},
    {"input": "Calculate the square of 5", "output": "square(5)"},
    {"input": "What's the cube of 2?", "output": "cube(2)"},
    {"input": "Greet John", "output": "greet('John')"}
]

# Additional diverse prompts
training_data.extend([
    {"input": "Add 10 and 15", "output": "add(10, 15)"},
    {"input": "Square of 6", "output": "square(6)"},
    {"input": "Cube the number 4", "output": "cube(4)"},
    {"input": "Say hello to Sarah", "output": "greet('Sarah')"}
])

In [8]:
# Use Mistral 7B as our base model
model_name = "mistralai/Mistral-7B-v0.1"

# Load model with quantization for efficiency
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Custom prompt engineering
def format_prompt(input_text):
    return f"""Given the user input, generate the corresponding function call:
Input: {input_text}
Function Call:"""

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
def generate_function_call(input_text, max_length=50):
    # Prepare prompt
    prompt = format_prompt(input_text)

    # Generate function call
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=False
    )

    # Decode and extract function call
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    function_call_match = re.search(r'Function Call:\s*(.+)', generated_text)

    if function_call_match:
        return function_call_match.group(1).strip()
    return None

def execute_function_call(function_call):
    try:
        # Extract function name and arguments
        match = re.match(r'(\w+)\((.*?)\)', function_call)
        if not match:
            return "Invalid function call format"

        func_name, args_str = match.groups()

        # Convert arguments
        args = []
        for arg in args_str.split(','):
            arg = arg.strip().strip('\'"')
            try:
                arg = int(arg) if arg.isdigit() else arg
            except ValueError:
                pass
            args.append(arg)

        # Execute function
        if func_name in FUNCTION_MAPPING:
            return FUNCTION_MAPPING[func_name](*args)
        else:
            return f"Unsupported function: {func_name}"

    except Exception as e:
        return f"Error executing function: {str(e)}"

In [10]:
def evaluate_model(test_cases):
    results = []
    for case in test_cases:
        function_call = generate_function_call(case['input'])
        expected_call = case['output']

        result = {
            'input': case['input'],
            'generated_call': function_call,
            'expected_call': expected_call,
            'call_match': function_call == expected_call
        }

        if result['call_match']:
            result['execution_result'] = execute_function_call(function_call)

        results.append(result)

    return results

In [11]:
# Test cases
test_cases = training_data + [
    {"input": "How much is 20 plus 22?", "output": "add(20, 22)"},
    {"input": "Square root of 16 squared", "output": "square(4)"},
    {"input": "Compute the cube of the square of 2", "output": "cube(square(2))"}  # Bonus challenge
]

evaluation_results = evaluate_model(test_cases)

# Print results
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    print(f"Call Match: {result['call_match']}")
    if result.get('execution_result'):
        print(f"Execution Result: {result['execution_result']}")
    print("---")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input: What is the sum of 7 and 3?
Generated Call: sum(7, 3)
Expected Call: add(7, 3)
Call Match: False
---
Input: Calculate the square of 5
Generated Call: square(5)
Expected Call: square(5)
Call Match: True
Execution Result: 25
---
Input: What's the cube of 2?
Generated Call: cube(2)
Expected Call: cube(2)
Call Match: True
Execution Result: 8
---
Input: Greet John
Generated Call: greet("John")
Expected Call: greet('John')
Call Match: False
---
Input: Add 10 and 15
Generated Call: add(10, 15)
Expected Call: add(10, 15)
Call Match: True
Execution Result: 25
---
Input: Square of 6
Generated Call: square(6)
Expected Call: square(6)
Call Match: True
Execution Result: 36
---
Input: Cube the number 4
Generated Call: cube(4)
Expected Call: cube(4)
Call Match: True
Execution Result: 64
---
Input: Say hello to Sarah
Generated Call: sayHello("Sarah")
Expected Call: greet('Sarah')
Call Match: False
---
Input: How much is 20 plus 22?
Generated Call: print(20 + 22)
Expected Call: add(20, 22)
Call 